In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pymysql
import tensorflow as tf
import pandas.io.sql as psql

In [10]:
con = pymysql.connect(host="18.218.142.199", user="root", passwd="111111", db="race", charset="utf8")
cursor = con.cursor()

In [ ]:
cursor = con.cursor()
ban_script = ("select * from ban limit 1000")
cure_script = ("select * from cure limit 1000")
grade_script = ("select * from grade limit 1000")
race_record_script = ("select * from race_record limit 1000")
race_test_script = ("select * from race_test limit 1000")
rider_script = ("select * from rider limit 1000")
rider_bu_script = ("select * from rider_bu limit 1000")
sudo_total_script = ("select * from sudo_total limit 1000")
total_hn_bu_script = ("select * from total_hn_bu limit 1000")
train_script = ("select * from train limit 1000")

ban = psql.read_sql(ban_script, con)
cure = psql.read_sql(cure_script, con)
grade_script = psql.read_sql(grade_script, con)
race_record = psql.read_sql(race_record_script, con)
race_test = psql.read_sql(race_test_script, con)
rider = psql.read_sql(rider_script, con)
rider_bu = psql.read_sql(rider_bu_script, con)
sudo_total = psql.read_sql(sudo_total_script, con)
total_hn_bu = psql.read_sql(total_hn_bu_script, con)
train = psql.read_sql(train_script, con)

con.close()